# **SETUP**

In [1]:

!pip install torch-geometric lightning wandb gymnasium
!pip install -U plotly
!pip install -U scikit-learn
!pip install -U kaleido
!pip install "notebook>=5.3" "ipywidgets>=7.5"
!mkdir Data
!mkdir Plots

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.9/800.9 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 48.8 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.15.0
    Uninstalling plotly-5.15.0:
      Successfully uninstalled plotly-5.15.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from torch.utils.data import Dataset, DataLoader
from torch_geometric.data import Data
import lightning as L
import torch
import torch.nn as nn
import wandb as wndb
from torch_geometric.nn import GATConv
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error
import numpy as np
from sklearn.model_selection import train_test_split
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
import gymnasium as gym
from gymnasium.envs.registration import register
import pdb
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# **MODEL AND RELATED STUFF**

In [3]:
class GreedyWorm(nn.Module):

    def __init__(self):
        super(GreedyWorm, self).__init__()

    def forward(self, data):
        graphs, positions = data
        return [graphs[i].x[positions[i]] for i in range(len(positions))]


In [4]:

class WormUpExamplesDataset(Dataset):

    def __init__(self, graphs: list[Data], actions: list[int], rewards: list[int]):
        self.data = list(zip(graphs, actions, rewards))

    def __getitem__(self, idx: int):
        return self.data[idx]

    def __len__(self):
        return len(self.data)

    def collate(self, data: list):
        graphs = []
        actions = []
        rewards = []
        for el in data:
            graphs.append(el[0])
            actions.append(el[1])
            rewards.append(el[2])
        return graphs, actions, torch.tensor(rewards, dtype=torch.float)

    def get_dataloader(self, batch_size: int, shuffle: bool = False):
        return DataLoader(self, batch_size=batch_size, shuffle=shuffle, collate_fn=self.collate)




In [5]:
class GraphNN(nn.Module):

    def __init__(self, in_size, out_size, h_size, deep, activation, device="cpu"):
        super(GraphNN, self).__init__()
        self.activation = activation
        if deep == 1:
            self.layers = [GATConv(in_size, out_size)]  #.to(device)]
        else:
            self.layers = [GATConv(in_size, h_size)]  #.to(device)]
            for _ in range(deep - 2):
                self.layers.append(GATConv(h_size, h_size))  #.to(device))
            self.layers.append(GATConv(h_size, out_size))  #.to(device))

    def forward(self, data):
        edge_index = data[1]
        x = data[0]
        for layer in self.layers[:-1]:
            x = self.activation(layer(x, edge_index))
        #breakpoint()
        return self.layers[-1](x, edge_index)



In [6]:
class LinearNN(nn.Module):
    def __init__(self, in_size, out_size, h_size, deep, activation):
        super(LinearNN, self).__init__()
        if deep == 1:
            layers = [nn.Linear(in_size, out_size), activation]
        else:
            layers = [nn.Linear(in_size, h_size), activation]
            for _ in range(deep - 2):
                layers.append(nn.Linear(h_size, h_size))
                layers.append(activation)
            layers.append(nn.Linear(h_size, out_size))
        self.linear = nn.Sequential(*layers)

    def forward(self, data):
        return self.linear(data)

In [17]:

class IntelligentWorm(L.LightningModule):

    def __init__(self, linear: nn.Module, gnn: nn.Module, lr: float = 1e-3):
        super(IntelligentWorm, self).__init__()
        self.encoder = gnn
        self.decoder = linear
        self.loss = nn.MSELoss()
        self.validation_predictions = []
        self.validation_targets = []
        self.validation_loss = []
        self.train_loss = []
        self.best_val_loss = 100000000
        self.best_mae = 100000000
        self.best_rmse = 1000000000
        self.best_r2 = -10000000
        self.best_model = 0
        self.lr = lr

    def update_best_stats(self, val_loss, mae, rmse, r2):
        self.best_val_loss = val_loss
        self.best_mae = mae
        self.best_rmse = rmse
        self.best_r2 = r2

    def forward(self, data):
        actions = data[1]
        graphs = data[0]
        embeddings = []
        for i in range(len(graphs)):
            g = graphs[i]
            x = self.encoder((g.x, g.edge_index))
            embeddings.append(x[actions[i]])
        embeddings = torch.stack(embeddings)
        pred = self.decoder(embeddings)
        return pred.squeeze()

    def training_step(self, batch, batch_idx):
        graphs, actions, rewards = batch
        predictions = self.forward((graphs, actions))
        train_loss = self.loss(predictions, rewards)
        self.train_loss.append(train_loss)

        return train_loss

    def validation_step(self, batch, batch_idx):
        graphs, actions, rewards = batch
        #breakpoint()
        if rewards == []:
          print("ERROR!!!!!!")
        self.validation_targets.append(rewards)
        predictions = self.forward((graphs, actions))
        validation_loss = self.loss(predictions, rewards)
        self.validation_predictions.append(predictions)
        self.validation_loss.append(validation_loss)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer



In [8]:
class WormCallback(L.Callback):

    def on_train_epoch_end(self, trainer, pl_module):

        epoch_mean = float(torch.stack(pl_module.train_loss).mean())
        print("training_epoch_mean loss = ", epoch_mean)
        # free up the memory
        pl_module.train_loss.clear()
        try:
            wndb.log({"train_loss": epoch_mean})
        except:
            pass

    def on_validation_epoch_end(self, trainer, pl_module: IntelligentWorm):
        #breakpoint()
        predictions = torch.cat(pl_module.validation_predictions, dim=0).squeeze()
        targets = torch.cat(pl_module.validation_targets, dim=0).squeeze()
        #
        r2 = r2_score(predictions, targets)
        mae = mean_absolute_error(predictions, targets)
        rmse = root_mean_squared_error(predictions, targets)
        mean_loss = float(torch.stack(pl_module.validation_loss).mean())

        pl_module.validation_loss.clear()
        pl_module.validation_predictions.clear()
        pl_module.validation_targets.clear()

        print("val_loss = ", mean_loss)
        print("mean_absolute_error = ", mae)
        print("root_mean_squared_error = ", rmse)
        print("r2 = ", r2)
        count = 0
        count += 1 if mean_loss < pl_module.best_val_loss else 0
        count += 1 if mae < pl_module.best_mae else 0
        count += 1 if rmse < pl_module.best_rmse else 0
        count += 1 if r2 > pl_module.best_r2 else 0
        if count >= 3 or count == 2 and mean_loss < pl_module.best_val_loss:
            pl_module.update_best_stats(mean_loss, mae, rmse, r2)
            pl_module.best_model -= 1
            pl_module.log("best_model", pl_module.best_model)
            gnn_state_dict = pl_module.encoder.state_dict()
            linear_state_dict = pl_module.decoder.state_dict()

            # Salva gli state dict su file
            torch.save(gnn_state_dict, "Model/gnn_checkpoint.pth")
            torch.save(linear_state_dict, "Model/linear_checkpoint.pth")
        else:
            pl_module.log("best_model", pl_module.best_model + 1)

        wndb.log({"val_loss": mean_loss, "mean_absolute_error": mae, "root_mean_squared_error": rmse, "r2": r2})




# **AGENT**

In [18]:
MODEL_VERSION = "model_V1_"
USE_SINGLE_WORM = False
USE_WORMHOLES = False

WORMHOLES = "wormholes_enabled_" if USE_WORMHOLES else "wormholes_disabled_"
WORMS = "single_worm_" if USE_SINGLE_WORM else "multi_worms_"



class WormsMasterAgent:

    def __init__(
            self,
            linear_part: nn.Module,
            gnn_part: nn.Module,
            initial_epsilon: float,
            epsilon_decay: float,
            final_epsilon: float,
            learning_rate: float = 1,
            discount_factor: float = 0.95,
            decay_after: int = 1,
            #trainer params
            batch_size: int = 8,
            episodes_for_batch: int = 20,
            trainer_deterministic: bool = True,
            trainer_max_epochs: int = 20,
            trainer_accelerator: str = "cpu"

    ):
        self.batch_size = batch_size
        self.episodes_for_batch = episodes_for_batch
        self.episode = 0
        self.linear = linear_part
        self.gnn = gnn_part
        self.learning_model = IntelligentWorm(linear_part, gnn_part)
        self.model = GreedyWorm()
        self.lr = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = initial_epsilon
        self.epsilon_decay = epsilon_decay
        self.final_epsilon = final_epsilon
        self.decay_after = decay_after
        self.checkpoint_callback = ModelCheckpoint(dirpath="Model/", filename="worms_model", save_top_k=1,
                                                   mode='min', monitor='best_model')
        self.worm_callback = WormCallback()
        self.early_stopping_callback = EarlyStopping(monitor='best_model', mode='min', patience=3)
        self.trainer = L.Trainer(deterministic=trainer_deterministic,
                                 max_epochs=trainer_max_epochs, accelerator=trainer_accelerator,
                                 callbacks=[self.checkpoint_callback, self.worm_callback, self.early_stopping_callback])
        self.actual_rewards = np.array([])
        self.actual_observations = []
        self.actual_actions = []
        self.model_training_data = {
            "actions": [],
            "observations": [],
            "rewards": []
        }

    def get_action(self, observation, available_actions: list[int]) -> int:
        graph = observation["field"]
        worms = observation["worms"]

        self.actual_rewards = np.append(self.actual_rewards, 0)
        if len(available_actions) == 1:
          return available_actions[0]
        # with probability epsilon return a random action to explore the environment
        if np.random.random() < self.epsilon:
            i = np.random.randint(0, high=len(available_actions))
            return available_actions[i]

        # with probability (1 - epsilon) act greedily (exploit)
        else:
            self.model.eval()
            with torch.no_grad():
                idx = np.argmax(self.model(([graph] * len(available_actions), available_actions)))
                return available_actions[idx]

    def update(self, observation, action: int, reward: int, terminated: bool):
        #breakpoint()
        graph = observation["field"]
        worms = observation["worms"]
        self.actual_observations.append(graph)
        self.actual_actions.append(action)
        self.actual_rewards += self.lr * reward
        episode_reward = 0.0
        len_worms_placed = 0.0
        if terminated:
            episode_reward = float(self.actual_rewards[0])
            len_worms_placed = len(self.actual_actions)
            print(f"total reward = {episode_reward}, actions list = {self.actual_actions}")
            self.episode += 1
            self.model_training_data["actions"] += self.actual_actions.copy()
            self.actual_actions = []
            self.model_training_data["observations"] += self.actual_observations.copy()
            self.actual_observations = []
            self.model_training_data["rewards"] += self.actual_rewards.tolist()
            self.actual_rewards = np.array([])
            if self.episode % self.decay_after == 0:
                self.decay_epsilon()
            print(f"starting episode {self.episode}, epsilon = {self.epsilon}")
            if self.episode % self.episodes_for_batch == 0:
                self.train_model()
                self.model = self.learning_model
        return episode_reward, len_worms_placed

    def decay_epsilon(self):
        self.epsilon = max(self.final_epsilon, self.epsilon - self.epsilon_decay)

    def prepare_data(self):
        #breakpoint()
        print("preparing data to train")
        x = list(zip(self.model_training_data["observations"], self.model_training_data["actions"]))
        y = self.model_training_data["rewards"]
        x_train, x_val, train_rewards, val_rewards = train_test_split(x, y, test_size=0.2)
        del x, y
        train_graphs = []
        train_actions = []
        for el in x_train:
            train_graphs.append(el[0])
            train_actions.append(el[1])
        val_graphs = []
        val_actions = []
        for el in x_val:
            val_graphs.append(el[0])
            val_actions.append(el[1])
        train_dataset = WormUpExamplesDataset(train_graphs, train_actions, train_rewards)
        val_dataset = WormUpExamplesDataset(val_graphs, val_actions, val_rewards)
        train_dataloader = train_dataset.get_dataloader(self.batch_size, shuffle=True)
        val_dataloader = val_dataset.get_dataloader(self.batch_size, shuffle=False)
        return train_dataloader, val_dataloader

    def train_model(self):
        print("begin training")
        train_dataloader, val_dataloader = self.prepare_data()
        wndb.init(
            # set the wandb project where this run will be logged
            project="WormsWarmingUp",
            name= MODEL_VERSION + WORMHOLES + WORMS +  "training_ep_" + str(self.episode),
            # track hyperparameters and run metadata
            config={
                "learning_rate": self.learning_model.lr,
                "architecture": str(self.learning_model),
                "batch": self.episode // self.episodes_for_batch
            }
        )
        self.trainer.fit(self.learning_model, train_dataloader, val_dataloader)
        wndb.finish()
        gnn_state_dict = torch.load("Model/gnn_checkpoint.pth")
        linear_state_dict = torch.load("Model/linear_checkpoint.pth")
        self.linear.load_state_dict(linear_state_dict)
        self.gnn.load_state_dict(gnn_state_dict)
        self.learning_model = IntelligentWorm.load_from_checkpoint(checkpoint_path="Model/worms_model.ckpt", linear=self.linear, gnn=self.gnn)
        print("finished training")


# **INITIALIZE MODEL, AGENT AND ENVIRONMENT**

In [25]:

# device = "gpu" if
gnn_part = GraphNN(1,64,0,1,nn.ReLU())
linear_part = LinearNN(64,1,128,2, nn.ReLU())




In [26]:

agent = WormsMasterAgent(linear_part, gnn_part,0.95,0.005,0.1)


INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [27]:
TABLE="00-example.txt"

register(
    id="worms_env",
    entry_point="worms_env:WormsEnv",
    max_episode_steps=300,
)
environment = gym.make('worms_env', env_file="Data/" + TABLE, render_mode="human", use_single_worm=USE_SINGLE_WORM, enable_wormholes=USE_WORMHOLES)



/usr/local/lib/python3.10/dist-packages/gymnasium/envs/registration.py:694: UserWarning:

WARN: Overriding environment worms_env already in registry.



In [13]:
wndb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [28]:
!rm -r Model
!mkdir Model

In [15]:
def compute_mean_rewards_to_print(rewards):
  #for x in rewards:
    # if x < 0:
    #   return 0
  return sum(rewards) / len(rewards)


# **TRAINING**

In [29]:
MAX_EPISODES = 200

ACCUMULATE = 5
accumulate_reward_means = []
accumulate_worm_len_means = []
last_rewards = [0 for _ in range(ACCUMULATE)]
last_len_worms_placed = [0 for _ in range(ACCUMULATE)]
done = False
obs, info = environment.reset()
print(f"starting episode {agent.episode}, epsilon = {agent.epsilon}")
# play one episode
while agent.episode < MAX_EPISODES:

    action = agent.get_action(info, environment.available_movements)
    obs, reward, terminated, truncated, info = environment.step(action)
    episode_reward, len_worms_placed = agent.update(info, action, reward, terminated)

    if terminated:
        obs, info = environment.reset()
        last_rewards[agent.episode % ACCUMULATE] = episode_reward
        last_len_worms_placed[agent.episode % ACCUMULATE] = len_worms_placed
        if agent.episode > ACCUMULATE:

          mean_reward = compute_mean_rewards_to_print(last_rewards)
          mean_len_worms_placed = sum(last_len_worms_placed) / len(last_len_worms_placed)
          accumulate_reward_means.append(mean_reward)
          accumulate_worm_len_means.append(mean_len_worms_placed)



/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:131: UserWarning:

WARN: The obs returned by the `reset()` method was expecting a numpy array, actual type: <class 'gymnasium.spaces.box.Box'>

/usr/local/lib/python3.10/dist-packages/gymnasium/spaces/box.py:240: UserWarning:

WARN: Casting input x to numpy array.

/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:159: UserWarning:

WARN: The obs returned by the `reset()` method is not within the observation space.



starting episode 0, epsilon = 0.95


/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:131: UserWarning:

WARN: The obs returned by the `step()` method was expecting a numpy array, actual type: <class 'gymnasium.spaces.box.Box'>

/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:159: UserWarning:

WARN: The obs returned by the `step()` method is not within the observation space.



total reward = 116.0, actions list = [2, 3, 4, 5, 6, 7, 47, 57, 58, 59, 49, 48, 38, 39, 29, 28, 27, 37, 19, 9, 8, 0, 1, 11]
starting episode 1, epsilon = 0.945
total reward = 111.0, actions list = [31, 32, 22, 23, 24, 14, 28, 18, 19, 9, 8, 7, 17, 50, 40, 30, 20, 10, 3, 2, 12, 6, 5, 15]
starting episode 2, epsilon = 0.94
total reward = 111.0, actions list = [15, 5, 4, 14, 24, 34, 37, 36, 46, 47, 57, 56, 55, 32, 31, 21, 22, 12, 29, 19, 9, 44, 43, 42]
starting episode 3, epsilon = 0.9349999999999999
total reward = 101.0, actions list = [39, 38, 28, 29, 19, 9, 35, 45, 44, 34, 33, 32, 42, 50, 51, 52, 53, 54, 55, 56, 57, 22, 23, 13]
starting episode 4, epsilon = 0.9299999999999999
total reward = 108.0, actions list = [17, 18, 8, 9, 19, 29, 26, 16, 15, 14, 24, 34, 33, 5, 4, 3, 13, 23, 2, 12, 22, 48, 49, 59]
starting episode 5, epsilon = 0.9249999999999999
total reward = -128.0, actions list = [23, 22, 12, 2, 1, 11, 28, 29, 39, 38, 37, 36, 46, 10, 0]
starting episode 6, epsilon = 0.91999999999

INFO: 
  | Name    | Type     | Params
-------------------------------------
0 | encoder | GraphNN  | 0     
1 | decoder | LinearNN | 8.4 K 
2 | loss    | MSELoss  | 0     
-------------------------------------
8.4 K     Trainable params
0         Non-trainable params
8.4 K     Total params
0.034     Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name    | Type     | Params
-------------------------------------
0 | encoder | GraphNN  | 0     
1 | decoder | LinearNN | 8.4 K 
2 | loss    | MSELoss  | 0     
-------------------------------------
8.4 K     Trainable params
0         Non-trainable params
8.4 K     Total params
0.034     Total estimated model params size (MB)


...

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:652: UserWarning:

Checkpoint directory /content/Model exists and is not empty.

INFO: 
  | Name    | Type     | Params
-------------------------------------
0 | encoder | GraphNN  | 0     
1 | decoder | LinearNN | 8.4 K 
2 | loss    | MSELoss  | 0     
-------------------------------------
8.4 K     Trainable params
0         Non-trainable params
8.4 K     Total params
0.034     Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name    | Type     | Params
-------------------------------------
0 | encoder | GraphNN  | 0     
1 | decoder | LinearNN | 8.4 K 
2 | loss    | MSELoss  | 0     
-------------------------------------
8.4 K     Trainable params
0         Non-trainable params
8.4 K     Total params
0.034     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

val_loss =  3385.259765625
mean_absolute_error =  44.44587
root_mean_squared_error =  58.182983
r2 =  -40.733637339631024


mean_absolute_error,▁
r2,▁
root_mean_squared_error,▁
val_loss,▁
mean_absolute_error,44.44587
r2,-40.73364
root_mean_squared_error,58.18298
val_loss,3385.25977


finished training


# **PLOT RESULTS OF TRAINING**

In [30]:


trace1 = go.Scatter(x=[i for i in range(len(accumulate_reward_means))], y=accumulate_reward_means, mode='lines', name="mean_rewards")


trace2 = go.Scatter(x=[i for i in range(len(accumulate_worm_len_means))], y=accumulate_worm_len_means, mode='lines', name="mean_lengths")

fig1 = go.Figure(trace1)
fig2 = go.Figure(trace2)






# Aggiunta del layout (opzionale)
fig1.update_layout(title='reward results', xaxis_title='X', yaxis_title='Y')



# Aggiunta del layout (opzionale)
fig2.update_layout(title='episode lengths results', xaxis_title='X', yaxis_title='Y')

fig1.write_image("Plots/rewards_" + MODEL_VERSION + WORMHOLES + WORMS[:-1] + ".png")
fig2.write_image("Plots/ep_lengths_" + MODEL_VERSION + WORMHOLES + WORMS[:-1] + ".png")

# Visualizzazione del grafico
fig1.show()

fig2.show()